<a href="https://colab.research.google.com/github/noushadkhan01/Basic-Mathematics-for-Machine-Learning/blob/master/Kaggle%20Competition/titanic%20survival/top_score_0_77990_till_now_by_decision_tree_classifier_recall_oriented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error,precision_score, recall_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [4]:
!ls

sample_data


In [6]:
!pip install kaggle

    100% |████████████████████████████████| 61kB 5.1MB/s 
    100% |████████████████████████████████| 235kB 14.5MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/8b/21/3b/a0076243c6ae12a6215b2da515fe06b539aee7217b406e510e
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify


In [0]:
from google.colab import files

In [9]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"noushadkhan","key":"fe4526bd4a314a15e1b0f4f2e43b0713"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

In [11]:
import kaggle

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [14]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 31.5MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 26.9MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 4.16MB/s]


In [15]:
train = pd.read_csv(r'train.csv')
train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [16]:
train.shape

(891, 12)

In [22]:
train.isnull().sum().sort_values(ascending = False)[:4]

Cabin       687
Age         177
Embarked      2
Fare          0
dtype: int64

In [18]:
test = pd.read_csv('test.csv')
test.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


In [19]:
test.shape

(418, 11)

In [23]:
test.isnull().sum().sort_values(ascending = False)[:4]

Cabin       327
Age          86
Fare          1
Embarked      0
dtype: int64

###### sns.pairplot(train.dropna(), hue = 'Survived')

## Data Warangling for both train and test data

In [24]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [0]:
def wrangling(df):
    df.drop(['Cabin', 'Name', 'Ticket', 'Embarked'], axis = 1, inplace = True)
    df = pd.get_dummies(df, drop_first = True)
    return df

In [26]:
train.shape

(891, 12)

In [27]:
train = wrangling(train)
train.shape

(891, 8)

In [28]:
train.head(2)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,1,0,3,22.0,1,0,7.2500,1
1,2,1,1,38.0,1,0,71.2833,0


In [29]:
train.shape

(891, 8)

In [30]:
sum(train.isnull().sum())

177

In [31]:
test = wrangling(test)
test.head(2)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,892,3,34.5,0,0,7.8292,1
1,893,3,47.0,1,0,7.0000,0


In [32]:
test.shape

(418, 7)

In [33]:
sum(test.isnull().sum())

87

## Data Visualisation for train data

In [34]:
corr = train.drop(['PassengerId'], axis = 1).corr().Survived.sort_values(ascending = False)
corr

Survived    1.000000
Fare        0.257307
Parch       0.081629
SibSp      -0.035322
Age        -0.077221
Pclass     -0.338481
Sex_male   -0.543351
Name: Survived, dtype: float64

# train:--fillAge

In [35]:
combined = pd.concat([train.drop(['Survived'], axis = 1), test], axis = 0)
combined.head(2)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,1,3,22.0,1,0,7.2500,1
1,2,1,38.0,1,0,71.2833,0


In [36]:
combined.shape

(1309, 7)

###### sns.jointplot(x = 'Age', y = 'Pclass', data = combined, kind = 'reg', #hue = 'sex'
                               )
plt.show()

In [37]:
combined.shape

(1309, 7)

## best fit line

In [0]:
from scipy.stats import linregress

In [39]:
len(combined[combined.Age == 80])

1

In [0]:
combined = combined[combined.Age != 80]

In [44]:
combined.shape

(1308, 7)

In [45]:
m, c, r, p, std = linregress(combined.dropna().Pclass, combined.dropna().Age)
m, c

(-6.9608020764237, 45.17224021662512)

### Mean of Age according to class

In [46]:
for i in range(1, 4):
    x = combined[combined.Pclass == i].Age.mean()
    print(f'class {i} mean is {x}')

class 1 mean is 39.0156183745583
class 2 mean is 29.506704980842912
class 3 mean is 24.81636726546906


## Age by Best fit line

In [47]:
for i in range(1, 4):
    x = m * i + c
    print(f'Age by class {i} is {x}')

Age by class 1 is 38.21143814020142
Age by class 2 is 31.25063606377772
Age by class 3 is 24.28983398735402


In [48]:
combined.shape

(1308, 7)

In [0]:
def fillAge(x, m, c):
    a, b = x[0], x[1]
    if np.isnan(a):
        return m * b + c
    return a

In [50]:
train['Age'] = train[['Age', 'Pclass']].apply(fillAge, args = (m, c), axis = 1)
train.head(2)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,1,0,3,22.0,1,0,7.2500,1
1,2,1,1,38.0,1,0,71.2833,0


In [51]:
train.shape

(891, 8)

In [52]:
test.shape

(418, 7)

## Train Mean after filling

In [53]:
for i in range(1, 4):
    x = train[train.Pclass == i].Age.mean()
    print(f'class {i} mean is {x}')

class 1 mean is 38.23038492687983
class 2 mean is 29.959711938595408
class 3 mean is 24.90496420016323


In [54]:
test['Age'] = test[['Age', 'Pclass']].apply(fillAge, args = (m, c), axis = 1)
test.head(2)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,892,3,34.5,0,0,7.8292,1
1,893,3,47.0,1,0,7.0000,0


In [55]:
test.shape

(418, 7)

## Test Age mean after filling

In [56]:
for i in range(1, 4):
    x = test[test.Pclass == i].Age.mean()
    print(f'class {i} mean is {x}')

class 1 mean is 40.69068171272722
class 2 mean is 28.910464304504178
class 3 mean is 24.114440582979313


In [57]:
train = train[train.Age != 80]
train.shape

(890, 8)

In [58]:
test.isnull().sum()

PassengerId    0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           1
Sex_male       0
dtype: int64

### test:-- fill Fare

In [59]:
sns.catplot(x = 'Pclass', y = 'Fare', data = train, kind = 'point')

AttributeError: ignored

In [61]:
null_columns=test.columns[test.isnull().any()]
test[test["Fare"].isnull()] #[null_columns]

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_male
152,1044,3,60.5,0,0,NaN,1


In [62]:
x = combined.Fare[combined.Pclass == 3]
len(x)

709

In [63]:
x.head(2)

0    7.250
2    7.925
Name: Fare, dtype: float64

In [64]:
x.describe()

count    708.000000
mean      13.302889
std       11.494358
min        0.000000
25%        7.750000
50%        8.050000
75%       15.245800
max       69.550000
Name: Fare, dtype: float64

In [65]:
test.fillna(13.302889, inplace = True)
sum(test.isnull().sum())

0

In [66]:
test.shape

(418, 7)

In [67]:
train.shape

(890, 8)

# MinMaxScaler()

### it is important for k nearest neighbors classifier

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train.iloc[:, :] = scaler.fit_transform(train.iloc[0:, :])
#train.plot(kind = 'box', figsize = (50, 50))

# outliers

###### plt.scatter(train.Fare, train.Survived)

train = train[train.Fare < .6]
train.shape

plt.scatter(train.Age, train.Survived)

train = train[train.Age < .8]
train.shape

## Train model

# LogisticRegression

In [0]:
sc2 = []
for i in range(1,50):
    lreg = LogisticRegression(C = float(i))
    lreg.fit(X_train, y_train)
    sc2.append(lreg.score(X_test, y_test))
    print(f'Train score is {lreg.score(X_train, y_train)}')
    print(f'Test score is {lreg.score(X_test, y_test)}')
    print(f'i is {i}')
    print(confusion_matrix(y_test, lreg.predict(X_test)))
    print(classification_report(y_test, lreg.predict(X_test)))
plt.figure(figsize = (15, 5))
plt.plot(range(1, 50), sc2, marker = 'o', linestyle = 'dashed')
plt.xticks(range(1, 50))
plt.show()

In [0]:
lreg = LogisticRegression(C = 4.0).fit(X_train, y_train)
print(f'Train score is {lreg.score(X_train, y_train)}')
print(f'Test score is {lreg.score(X_test, y_test)}')

In [0]:
lc = confusion_matrix(y_test, lreg.predict(X_test))
lc

In [0]:
print(classification_report(y_test, lreg.predict(X_test)))

In [0]:
lreg = LogisticRegression(C = 4.0).fit(X, y)
cv_score = cross_val_score(lreg, X, y, cv = 10)
lm = np.mean(cv_score)
lm

# DecisionTreeClassifier

In [0]:
from sklearn.tree import DecisionTreeClassifier

In [0]:
n = 20
while n > 1:
    Tmodel = DecisionTreeClassifier(max_depth = n, random_state = 0).fit(X_train, y_train)
    print(f'depth is {n}')
    print(f'Train score is {Tmodel.score(X_train, y_train)}')
    print(f'Test score is {Tmodel.score(X_test, y_test)}\n')
    print(confusion_matrix(y_test, Tmodel.predict(X_test)))
    n -= 1

In [0]:
Tmodel = DecisionTreeClassifier(random_state = 0, max_depth = 13).fit(X_train, y_train)

In [0]:
dc = confusion_matrix(y_test, Tmodel.predict(X_test))
dc

In [0]:
print(classification_report(y_test, Tmodel.predict(X_test)))

In [0]:
Tmodel = DecisionTreeClassifier(max_depth = 4).fit(X, y)
cv_score = cross_val_score(Tmodel, X, y, cv = 10)
dm = np.mean(cv_score)
dm

# By RandomForest

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
train.shape

In [0]:
len(train[train.Survived == 0])

In [0]:
len(train[train.Survived == 1])

In [0]:
n = 20
while n > 0:
    Rmodel = RandomForestClassifier(random_state = 0, max_depth = n).fit(X_train, y_train)
    print(f'Max depth is {n}')
    print(f'Train score is {Rmodel.score(X_train, y_train)}')
    print(f'Test score is {Rmodel.score(X_test, y_test)}')
    print(confusion_matrix(y_test, Rmodel.predict(X_test)))
    n -= 1

In [0]:
Rmodel = RandomForestClassifier(random_state = 0, max_depth = 20).fit(X_train, y_train)

In [0]:
rc = confusion_matrix(y_test, Rmodel.predict(X_test))
rc

In [0]:
print(classification_report(y_test, Rmodel.predict(X_test)))

In [0]:
mean = []
for i in range(1, 21):
    Rmodel = RandomForestClassifier(random_state = 0, max_depth = i).fit(X, y)
    cv_score = cross_val_score(Rmodel, X, y, cv = 10)
    rm = np.mean(cv_score)
    mean.append(rm)
plt.plot(range(1, 21), mean)
plt.xticks(range(1, 21))
plt.show()

In [0]:
Rmodel = RandomForestClassifier(random_state = 0, max_depth = 20).fit(X, y)
cv_score = cross_val_score(Rmodel, X, y, cv = 10)
rm = np.mean(cv_score)
rm

# support vector machines

In [0]:
from sklearn.svm import SVC

In [0]:
model = SVC(kernel = 'linear').fit(X_train, y_train)
print(f'Train score is {model.score(X_train, y_train)}')
print(f'Test score is {model.score(X_test, y_test)}')
print(confusion_matrix(y_test, model.predict(X_test)))
print(classification_report(y_test, model.predict(X_test1)))

In [0]:
print(kc, km)
print('\n')
print(lc, lm)
print('\n')
print(dc, dm)
print('\n')
print(rc, rm)

## For test data

In [0]:
test.head(3)

In [0]:
test.shape

In [0]:
sum(test.isnull().sum())

In [0]:
PassengerId = test['PassengerId']
PassengerId.head(2)

In [0]:
submission = pd.DataFrame()
submission.head(2)

In [0]:
submission['PassengerId'] = PassengerId
submission.head(2)

In [0]:
xtest = test.drop(['PassengerId'], axis = 1)
xtest.head(2)

###### xtest.iloc[:, :] = MinMaxScaler().fit_transform(xtest.iloc[0:, :])

In [0]:
xtest.head(2)

In [0]:
Tmodel

In [0]:
y_predicted2 = Tmodel.predict(xtest)
y_predicted2[:10]

In [0]:
submission['Survived'] = y_predicted2
submission.head(2)

In [0]:
submission.to_csv('train 2 oct sept by decision tree recall oriented depth is 4.csv', index = False)

###### X_train.to_csv('X_train.csv', index = False)
X_test.to_csv('X_test.csv', index = False)
y_train = pd.DataFrame(y_train)
y_train.to_csv('y_train.csv', index = False)
y_test = pd.DataFrame(y_test)
y_test.to_csv('y_test.csv', index = False)

###### from sklearn.externals import joblib

joblib.dump(model2, 'logistic kaggle 0.7606 highest 18september.pkl')

model3 = joblib.load('logistic kaggle 0.7606 highest 18september.pkl')

model3.score(X_test, y_test)